<a href="https://www.kaggle.com/code/zokovik/cifar-zokovik?scriptVersionId=187286728" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# importing necesssary libraries

import os
import pandas as pd
import torch
import torch.nn as nn

In [3]:
# Building test and train CSV file, which contains details like image_path, class_name, etc.,. Can be used by Dataset class. 

def build_csv(data, csv_name):
    import csv
    
    with open(csv_name, "w", newline = "") as csvfile:
        writer = csv.writer(csvfile, delimiter = ',')
        writer.writerow(["file_name", "file_path", "class_name", "class_index"])
        for idx, partition in enumerate(os.listdir(data)):
            file = os.path.join(data, partition)
            for f in os.listdir(file):
                file_path = os.path.join(file, f)
                writer.writerow([f, file_path, partition, idx])
    
       
    return
    
            
    
train_data_loc = "/kaggle/input/cifar10/cifar10/train"
test_data_loc = "/kaggle/input/cifar10/cifar10/test"
build_csv(train_data_loc, "train_csv")
build_csv(test_data_loc, "validation_csv")

In [4]:
# loading the test and train csv files

df_test = pd.read_csv("train_csv")
df_val = pd.read_csv("validation_csv")

In [5]:
df_test

,file_name,file_path,class_name,class_index
0,29606_airplane.png,/kaggle/input/cifar10/cifar10/train/airplane/2...,airplane,0
1,23684_airplane.png,/kaggle/input/cifar10/cifar10/train/airplane/2...,airplane,0
2,46324_airplane.png,/kaggle/input/cifar10/cifar10/train/airplane/4...,airplane,0
3,16513_airplane.png,/kaggle/input/cifar10/cifar10/train/airplane/1...,airplane,0
4,40807_airplane.png,/kaggle/input/cifar10/cifar10/train/airplane/4...,airplane,0
...,...,...,...,...
49995,41608_deer.png,/kaggle/input/cifar10/cifar10/train/deer/41608...,deer,9
49996,6785_deer.png,/kaggle/input/cifar10/cifar10/train/deer/6785_...,deer,9
49997,17777_deer.png,/kaggle/input/cifar10/cifar10/train/deer/17777...,deer,9
49998,9202_deer.png,/kaggle/input/cifar10/cifar10/train/deer/9202_...,deer,9


In [6]:
# Our Dataset Class 

from PIL import Image
from torch.utils.data import Dataset
class HHDataset(Dataset):
    def __init__(self, csv_file,transforms = None):
        self.csv_file = pd.read_csv(csv_file)
        self.transforms = transforms
        self.len = len(self.csv_file)
    def __getitem__(self, index):
                                    
        image = Image.open(self.csv_file.iloc[index][1])
        image = Image.open(self.csv_file.iloc[index][1]).convert("RGB")
        if self.transforms:
            image = self.transforms(image)
        return image, self.csv_file.iloc[index][3]
        
    def __len__(self):
        return self.len

In [7]:
# lets use some failry simple transforms 

from torchvision import transforms

# Define a transformation (if needed)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [8]:
""" Defining our model, which consists of 3 CNN layers, each followed by a Relu activation and max-pooling. We then flatten out the resultant matrix, and 
 pass it through a fully connected layer. 
"""

import torch.nn as nn
class Cmodel(nn.Module):
    def __init__(self):
        super(Cmodel, self).__init__()
        
        self.model = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 64, stride = 1, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
    ,nn.Conv2d(in_channels = 64, out_channels = 128, stride = 1, kernel_size=3)
            ,nn.ReLU()
      ,nn.MaxPool2d(kernel_size = 3, stride = 2)
            ,nn.Conv2d(in_channels = 128, out_channels = 256, stride = 1, kernel_size=3)
            ,nn.ReLU()
        ,nn.MaxPool2d(kernel_size = 3, stride = 2),
        )
        
        self.linear = nn.Sequential(nn.Linear(3328* 13, 20)
       ,nn.Linear(20,10),
#                                    nn.Softmax()
                                   )
    def forward(self, x):
        out = self.model(x)
        out = out.view(-1, 3328* 13)
        out = self.linear(out)
        return out

In [9]:
# checking out, if it works as desired 

x = torch.randn(3,128,128)
c = Cmodel()
res = c(x)
res

tensor([[-0.2872, -0.1906, -0.0215, -0.0282,  0.2961, -0.0911,  0.1311, -0.1978,
         -0.2662,  0.1041]], grad_fn=<AddmmBackward0>)

In [10]:
# creating the data loaders for training data and test data

from torch.utils.data import DataLoader
train_data = HHDataset(csv_file = "/kaggle/working/train_csv", transforms = transform )
val_data = HHDataset(csv_file = "/kaggle/working/validation_csv", transforms = transform)


trainLoader = DataLoader(train_data,batch_size = 512, shuffle = True)
testLoader = DataLoader(val_data, batch_size = 512, shuffle = True)


In [11]:
# trainging our model.

epochs = 40
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
model = Cmodel().to(device)

optimizer = optim.Adam(model.parameters(), lr = 0.001)



for epoch in range(epochs):
  
    model.train()
    
    correct = total = 0
    for batch_size, (data, target) in enumerate(trainLoader):
        
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        target = target.float()
        output = model(data)

        output = output.squeeze()
        
        target = target.to(torch.int64)

        # notice that, Cross entropy loss function, by itself converts the outputs to softmax for prediction, and also converts the target to one-hot
#         encoded values
        loss = criterion(output, target)
        
        

        loss.backward()
        
        optimizer.step()

        total += target.size(0)
        
        predicted_classes = torch.argmax(output, dim=1)
        
        correct += (predicted_classes == target).sum().item()
        
        
        if (batch_size % 30 == 0):
            print(loss, "=================")
            print(correct, "correct")
    
    
    
    print(f"epoch{epoch} completed", correct/total)
    
        

cuda


/tmp/ipykernel_25/3604617531.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image = Image.open(self.csv_file.iloc[index][1])
/tmp/ipykernel_25/3604617531.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image = Image.open(self.csv_file.iloc[index][1]).convert("RGB")
/tmp/ipykernel_25/3604617531.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return image, self.csv_file.iloc[index][3]


tensor(2.3075, device='cuda:0', grad_fn=<NllLossBackward0>) =================
50 correct
tensor(1.8888, device='cuda:0', grad_fn=<NllLossBackward0>) =================
3511 correct
tensor(1.6278, device='cuda:0', grad_fn=<NllLossBackward0>) =================
9315 correct
tensor(1.6142, device='cuda:0', grad_fn=<NllLossBackward0>) =================
15751 correct
epoch0 completed 0.34492
tensor(1.5937, device='cuda:0', grad_fn=<NllLossBackward0>) =================
224 correct
tensor(1.5132, device='cuda:0', grad_fn=<NllLossBackward0>) =================
7301 correct
tensor(1.3900, device='cuda:0', grad_fn=<NllLossBackward0>) =================
14776 correct
tensor(1.4628, device='cuda:0', grad_fn=<NllLossBackward0>) =================
22401 correct
epoch1 completed 0.4817
tensor(1.2806, device='cuda:0', grad_fn=<NllLossBackward0>) =================
276 correct
tensor(1.3394, device='cuda:0', grad_fn=<NllLossBackward0>) =================
8329 correct
tensor(1.2686, device='cuda:0', grad_fn=<N

In [12]:
# testing our validation accuracy


with torch.no_grad():
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data, labels in testLoader:

            data, labels = data.to(device), labels.to(device)
            outputs = model(data)

            outputs = outputs.squeeze()
#                 predicted = (outputs > 0.5).int()

            total += labels.size(0)
            predicted_classes = torch.argmax(outputs, dim=1)

            correct += (predicted_classes == labels).sum().item()



#         print(correct, total)
    accuracy = correct / total
    print(f"val accuracy {accuracy}")

/tmp/ipykernel_25/3604617531.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image = Image.open(self.csv_file.iloc[index][1])
/tmp/ipykernel_25/3604617531.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image = Image.open(self.csv_file.iloc[index][1]).convert("RGB")
/tmp/ipykernel_25/3604617531.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return image, self.csv_file.iloc[index][3]


val accuracy 0.6729
